In [1]:
# 
import os
import concurrent
import queue
import collections
import json
# 
import openpyxl
import urllib
import requests
import pendulum
import tqdm
import matplotlib
import matplotlib.figure
import matplotlib.backends.backend_agg
import numpy
# 
import furtive
import cleats
import colormaps

/Users/brianpetersen/.pyenv/versions/3.6.1/envs/achilles/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator KNeighborsRegressor from version pre-0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/brianpetersen/.pyenv/versions/3.6.1/envs/achilles/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator KNeighborsRegressor from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [2]:
HOST = 'https://www.podimetrics.com'

In [3]:
class PodimetricsConnectionPool:
    
    sessions = queue.Queue()

    @classmethod
    def get(self, *args, **kwargs):
        # create session if one doesn't exist
        try:
            session = self.sessions.get(block=False)
        except:
            session = requests.Session()
            response = session.post(
                urllib.parse.urljoin(HOST, 'api/v1/sessions/'),
                json={
                    'user_id': furtive.podimetrics.accounts['superuser'].user_id,
                    'password': furtive.podimetrics.accounts['superuser'].password,
                },
            )
        # attempt request, if fails don't put back session
        try:
            response = session.get(*args, **kwargs)
            if response.status_code == 403:
                return self.get(*args, **kwargs)
            response.raise_for_status()
        except:
            return self.get(*args, **kwargs)
        if self.sessions.qsize() < 10:
            self.sessions.put(session)
        return response


In [4]:
class ApiBase:
    
    connection_pool = PodimetricsConnectionPool()

In [5]:
"""
def setup_connection_pool(user_id, password, threads=1):
    ApiBase.connection_pool = PodimetricsConnectionPool(
        account.user_id,
        account.password,
        threads=threads,
    )
    return ApiBase.connection_pool
"""

'\ndef setup_connection_pool(user_id, password, threads=1):\n    ApiBase.connection_pool = PodimetricsConnectionPool(\n        account.user_id,\n        account.password,\n        threads=threads,\n    )\n    return ApiBase.connection_pool\n'

In [6]:
class ApiDevice(ApiBase):
    
    pass


class ApiScan(ApiBase):
    
    def __init__(self, api_scan):
        self.api_scan = api_scan
        self.cache = {}
    
    @property
    def id(self):
        return self.api_scan['scan_id']
    
    @property
    def type(self):
        return 'diagnostic' if self.api_scan['mat_thermogram_url'] is None else 'actuated'
    
    @property
    def footness(self):
        if 'footness' not in self.cache:
            left_footness = self.api_scan['left_footness']
            right_footness = self.api_scan['right_footness']
            if left_footness is None or right_footness is None:
                self.cache['footness'] = None
            else:
                self.cache['footness'] = min(left_footness, right_footness)
        return self.cache['footness']
        
    @property
    def when_completed(self):
        return pendulum.parse(self.api_scan['when_scan_completed'])
    
    @property
    def feet_thermograms(self):
        url = self.api_scan['feet_thermogram_url']
        response = self.connection_pool.get(url)
        thermograms = response.json()
        return (thermograms['left_foot_thermogram'], thermograms['right_foot_thermogram'])
    
    @property
    def raw_measurement_thermograms(self):
        url = self.api_scan['raw_thermogram_url']
        response = self.connection_pool.get(url)
        measurements = response.json()['measurements']
        return measurements
    
    @property
    def mat_thermogram(self):
        url = self.api_scan['mat_thermogram']
        response = self.connection_pool.get(url)
        thermogram = response.json()['thermogram']
        return thermogram
    
    def __repr__(self):
        return '<scan {} at {}>'.format(self.id, self.when_completed)
    
    def __hash__(self):
        return hash('scan', self.id)


class ApiScans(ApiBase):
    
    def __init__(self, scans):
        self.scans = scans
        self.scans_by_id = {}
        for scan in self.scans:
            self.scans_by_id[scan.id] = scan
    
    def sort_by_date(self, ascending=False):
        reverse = not ascending
        self.scans.sort(key=lambda scan: scan.when_completed, reverse=reverse)
    
    def sort_by_footness(self, ascending=False):
        reverse = not ascending
        self.scans.sort(key=lambda scan: scan.footness, reverse=reverse)
    
    def filter_by_date(self, date):
        after = date.replace(hour=0, minute=0, second=0, microsecond=0)
        before = date.add(days=1)
        return self.filter_between(after, before, after_inclusive=True, before_inclusive=True)
            
    def filter_by_type(self, type):
        scans = []
        for scan in self.scans:
            if scan.type == type:
                scans.append(scan)
        return self.__class__(scans)
    
    def filter_between(self, after, before, after_inclusive=True, before_inclusive=False):
        if after_inclusive:
            is_after = lambda scan: scan.when_completed >= after
        else:
            is_after = lambda scan: scan.when_completed > after
        if before_inclusive:
            is_before = lambda scan: scan.when_completed <= before
        else:
            is_before = lambda scan: scan.when_completed < before
        scans = []
        for scan in self.scans:
            if is_after(scan) and is_before(scan):
                scans.append(scan)
        return self.__class__(scans)
    
    def after(self, when, inclusive=True):
        return self.filter_between(
            after=when,
            before=pendulum.max,
            after_inclusive=inclusive,
            before_inclusive=True,
        )
        
    def before(self, when, inclusive=True):
        return self.filter_between(
            after=pendulum.min,
            before=when,
            after_inclusive=True,
            before_inclusive=inclusive,
        )
    
    def filter_by_footness(self, lower, upper=None, lower_inclusive=True, upper_inclusive=False):
        scans = []
        for scan in self.scans:
            if scan.footness is not None and is_footness_less(scan) and is_footness_greater(scan):
                scans.append(scan)
        return self.__class__(scans)
    
    @property
    def first(self):
        return min(self.scans, key=lambda scan: scan.when_completed, default=None)
    
    @property
    def last(self):
        return max(self.scans, key=lambda scan: scan.when_completed, default=None)
    
    def __iter__(self):
        for scan in self.scans:
            yield scan
    
    def __contains__(self, scan):
        if isinstance(scan, self.__class__):
            scan_id = scan.id
        else:
            scan_id = scan
        return scan_id in self.scans_by_id
    
    def __len__(self):
        return len(self.scans)
    
    def __getitem__(self, scan_id):
        return self.scans_by_id[scan_id]
    
    def __repr__(self):
        return repr(list(self))


class ApiPatient(ApiBase):
    
    primary_affiliations = frozenset((
        'Humana',
        'VA Phoenix',
        'VA Tucson',
        'VA Grand Junction',
        'VA Salt Lake City',
        'VA Los Angeles',
    ))
    
    def __init__(self, api_patient):
        self.api_patient = api_patient
    
    @property
    def device_id(self):
        return self.api_patient['serial_number']
    
    @property
    def id(self):
        return self.api_patient['patient_id']
    
    @property
    def name(self):
        api_patient = self.api_patient
        return '{} {}'.format(api_patient['first_name'], api_patient['last_name']).strip()
    
    @property
    def scans(self):
        scans = getattr(self, '_scans', None)
        if scans is None:
            scans_url = self.api_patient['scans_url']
            connection_pool = self.connection_pool
            api_scans = connection_pool.get(scans_url).json()['scans']
            self._scans = scans = ApiScans([ApiScan(api_scan) for api_scan in api_scans])
        return scans
    
    @property
    def affiliation(self):
        """Primary affiliation"""
        for affiliation in self.primary_affiliations:
            if affiliation in self.affiliations:
                return affiliation
    
    @property
    def affiliations(self):
        """All affiliations"""
        return set(self.api_patient['affiliations'])
    
    def __repr__(self):
        return '<patient {}>'.format(self.name)
    
    def __hash__(self):
        return hash('patient', self.id)

    
class ApiPatients(ApiBase):
    
    def __init__(self, patients):
        self.patients = patients
        self.patients_by_id = {}
        for patient in self.patients:
            self.patients_by_id[patient.id] = patient
    
    @classmethod
    def from_api(cls):
        response = cls.connection_pool.get(urllib.parse.urljoin(HOST, 'api/v1/patients/'))#'https://www.podimetrics.com/api/v1/patients/')
        api_patients = response.json()['patients']
        patients = [ApiPatient(api_patient) for api_patient in api_patients]
        return cls(patients)
    
    @property
    def patient_ids(self):
        for patient in self.patients:
            yield patient.id
        
    def filter_by_affiliations(self, *affiliations):
        affiliations = set(affiliations)
        patients = []
        for patient in self:
            if patient.affiliation in affiliations:
                patients.append(patient)
        return self.__class__(patients)
        
    def __iter__(self):
        return iter(self.patients)
    
    def __len__(self):
        return len(self.patients)
    
    def __getitem__(self, index):
        return self.patients[index]
    
    def __contains__(self, patient):
        return patient in self.patients
        
    def __repr__(self):
        return repr(list(self))

In [7]:
class ApiPatientAdherence:
    
    def __init__(
            self, patient, after, before=None, threshold_for_likely_patient_scan=0.7,
            lower_threshold_for_review=0.5, upper_threshold_for_review=0.9,
        ):
        self.patient = patient
        self.after = after
        if before is None:
            before = after.add(days=1)
        self.before = before
        self.scans = patient.scans
        self.threshold_for_likely_patient_scan = threshold_for_likely_patient_scan
        self.lower_threshold_for_review = lower_threshold_for_review
        self.upper_threshold_for_review = upper_threshold_for_review
    
    @property
    def actuated_scans(self):
        if not hasattr(self, '_actuated_scans'):
            self._actuated_scans = self.scans.filter_by_type('actuated').filter_between(
                self.after, self.before,
            )
        return self._actuated_scans
    
    @property
    def diagnostic_scans(self):
        if not hasattr(self, '_diagnostic_scans'):
            self._diagnostic_scans = self.scans.filter_by_type('diagnostic').filter_between(
                self.after, self.before,
            )
        return self._diagnostic_scans
    
    @property
    def patient_id(self):
        return self.patient.id
    
    @property
    def actuated_scan_ids(self):
        return [scan.id for scan in self.actuated_scans]
    
    @property
    def has_actuated_scan(self):
        return len(self.actuated_scans) > 0
    
    @property
    def has_diagnostic_scan(self):
        return len(self.diagnostic_scans) > 0
    
    @property
    def days_since_most_recent_actuated_scan(self):
        all_actuated_scans = self.scans.filter_by_type('actuated')
        if len(all_actuated_scans) == 0:
            return None
        else:
            when = all_actuated_scans.last.when_completed
            return (self.before - when).days
    
    @property
    def days_since_most_recent_diagnostic_scan(self):
        all_diagnostic_scans = self.scans.filter_by_type('diagnostic')
        if len(all_diagnostic_scans) == 0:
            return None
        else:
            when = all_diagnostic_scans.last.when_completed
            return (self.before - when).days
    
    @property
    def footness(self):
        scans = self.actuated_scans
        if len(scans) == 0:
            return None
        else:
            footnesses = (scan.footness for scan in scans if scan.footness is not None)
            return max(footnesses, default=None)
    
    @property
    def review_required(self):
        if self.has_actuated_scan:
            if self.footness is None:
                return True
            else:
                lower_threshold = self.lower_threshold_for_review
                upper_threshold = self.upper_threshold_for_review
                return lower_threshold <= self.footness <= upper_threshold
        else:
            return False

    @property
    def has_likely_patient_scan(self):
        if self.has_actuated_scan:
            if self.footness is None:
                return None
            else:
                return self.footness >= self.threshold_for_likely_patient_scan
        else:
            return False
    
    @property
    def status(self):
        if self.review_required:
            return '-'
        if self.has_actuated_scan:
            if self.has_likely_patient_scan == True:
                return True
            elif self.has_likely_patient_scan == False:
                return False
            else:
                raise Exception()
        else:
            return False
    
    @property
    def sort_key(self):
        footness = self.footness
        if footness is None and self.review_required:
            footness = self.threshold_for_likely_patient_scan
        return (
            not self.review_required,
            abs(footness - self.threshold_for_likely_patient_scan) if footness is not None else 2,
            self.patient.name
        )
    
    def __repr__(self):
        if self.footness is None:
            footness = self.footness
        else:
            footness = round(self.footness, 2)
        if (self.before - self.after).total_days() == 1.0:
            return '<patient adherence for {} on {}: footness = {}>'.format(
                self.patient.name,
                self.after.date(),
                footness,
            )
        else:
            return '<patient adherence for {} between {} and {}: footness = {}>'.format(
                self.patient.name,
                self.after,
                self.before,
                footness,
            )

In [8]:
THIN_BORDER = openpyxl.styles.Side(
    border_style='thin',
    color='FF000000',
)
HIGHLIGHT_FILL = openpyxl.styles.PatternFill(
    start_color='FFFFE000',
    end_color='FFFFE000',
    fill_type='solid',
)
ADHERENCE_VALIDATION = openpyxl.worksheet.datavalidation.DataValidation(
    type='list',
    formula1='"True, False, -"',
)

In [9]:
class RawThermogramWriter:
    
    def __init__(self, measurements, dpi=25):
        if numpy.shape(measurements[-1]) == (36, 54):
            thermogram = numpy.ma.masked_where(cleats.masks[6], measurements[-1])
        else:
            thermogram = numpy.ma.masked_where(cleats.masks[7], measurements[-1])
        self.figure = figure = matplotlib.figure.Figure(frameon=False, dpi=dpi)
        canvas = matplotlib.backends.backend_agg.FigureCanvasAgg(figure)
        axis = figure.add_axes([0, 0, 1, 1])
        axis.axis('off')
        axis.imshow(thermogram, interpolation='nearest', cmap=colormaps.greys)
    
    def save(self, filename):
        self.figure.canvas.print_png(filename)


class PatientAdherenceWriter:
    
    headers = (
        'Date', 'Patient Name', 'Patient ID', 'Affiliation', 'Adherence Status', 
        'Thermograms', 'Footness', 'Serial Number', 'Last Diagnostic (Days)', 
        'Review Required', 'Scan IDs',
    )
    
    def __init__(
            self, patients, date, workbook_password, thermogram_directory=None, 
            threshold_for_likely_patient_scan=0.7, lower_threshold_for_review=0.5,
            upper_threshold_for_review=0.9,
        ):
        self.date = date.replace(hour=0, minute=0, second=0, microsecond=0)
        self.workbook_password = workbook_password
        if thermogram_directory is None:
            thermogram_directory = '.thermograms/adherence/'
        self.thermogram_directory = thermogram_directory
        self.ensure_directory_exists(self.thermogram_directory)
        self.patients = patients
        self.threshold_for_likely_patient_scan = threshold_for_likely_patient_scan
        self.lower_threshold_for_review = lower_threshold_for_review
        self.upper_threshold_for_review = upper_threshold_for_review

    def populate_workbook(self, progress=True):
        self.setup_workbook()
        self.write_headers()
        self.setup_columns()
        self.write_rows(progress)
    
    def ensure_directory_exists(self, directory):
        os.makedirs(directory, exist_ok=True)
    
    @property
    def patients_adherence(self):
        if not hasattr(self, '_patients_adherence'):
            self.cache_patients_adherence()
        return self._patients_adherence
    
    def cache_patients_adherence(self, progress=False):
        self._patients_adherence = patients_adherence = []
        for patient in tqdm.tqdm(self.patients):
            patient_adherence = ApiPatientAdherence(
                patient,
                after=self.date,
                before=None,
                threshold_for_likely_patient_scan=self.threshold_for_likely_patient_scan,
                lower_threshold_for_review=self.lower_threshold_for_review,
                upper_threshold_for_review=self.upper_threshold_for_review,
            )
            patients_adherence.append(patient_adherence)
        patients_adherence.sort(key=lambda adherence: adherence.sort_key)
    
    def write_rows(self, progress):
        self.cache_patients_adherence(progress)
        for index, adherence in enumerate(self.patients_adherence):
            row_index = index + 2
            self.write_and_format_row(row_index, adherence)
    
    def setup_columns(self):
        self.sheet.column_dimensions['C'].hidden= True
        self.sheet.column_dimensions['K'].hidden= True
        self.sheet.freeze_panes = self.sheet.cell('E2')
    
    def write_and_format_row(self, row_index, adherence):
        if adherence.review_required:
            style = openpyxl.styles.Style(
                fill=HIGHLIGHT_FILL,
                alignment=openpyxl.styles.Alignment(horizontal='left'),
            )
            dropdown_style = openpyxl.styles.Style(
                fill=HIGHLIGHT_FILL,
                protection=openpyxl.styles.Protection(locked=False),
                alignment=openpyxl.styles.Alignment(horizontal='center'),
            )
        else:
            style = openpyxl.styles.Style(
                alignment=openpyxl.styles.Alignment(horizontal='left'),
            )
            dropdown_style = openpyxl.styles.Style(
                protection=openpyxl.styles.Protection(locked=False),
                alignment=openpyxl.styles.Alignment(horizontal='center'),
            )
        #
        cell = self.sheet.cell(row=row_index, column=1)
        cell.style = style
        cell.value = self.date.date()
        #
        cell = self.sheet.cell(row=row_index, column=2)
        cell.style = style
        cell.value = adherence.patient.name
        #
        cell = self.sheet.cell(row=row_index, column=3)
        cell.style = style
        cell.value = adherence.patient.id
        #
        cell = self.sheet.cell(row=row_index, column=4)
        cell.style = style
        cell.value = adherence.patient.affiliation
        #
        cell = self.sheet.cell(row=row_index, column=5)
        cell.style = dropdown_style
        cell.value = adherence.status
        ADHERENCE_VALIDATION.add(cell)
        #
        cell = self.sheet.cell(row=row_index, column=6)
        cell.style = style
        cell.value = '=HYPERLINK("{}", "{} thermogram{}")'.format(
            os.path.join(
                self.thermogram_directory,
                adherence.patient.id,
                self.date.format('YYYY-MM-DD', formatter='alternative'),
            ),
            len(adherence.actuated_scans),
            '' if len(adherence.actuated_scans) == 1 else 's',
        )
        #
        cell = self.sheet.cell(row=row_index, column=7)
        cell.style = style
        cell.value = adherence.footness
        cell.number_format = '0.00'
        #
        cell = self.sheet.cell(row=row_index, column=8)
        cell.style = style
        cell.value = adherence.patient.device_id
        #
        cell = self.sheet.cell(row=row_index, column=9)
        cell.style = style
        cell.value = adherence.days_since_most_recent_diagnostic_scan
        #
        cell = self.sheet.cell(row=row_index, column=10)
        cell.style = style
        cell.value = adherence.review_required
        #
        cell = self.sheet.cell(row=row_index, column=11)
        cell.style = style
        cell.value = json.dumps(adherence.actuated_scan_ids)
    
    def setup_workbook(self):
        self.workbook = workbook = openpyxl.Workbook()
        self.sheet = sheet = workbook.active
        sheet.title = 'Adherence'
        sheet.protection.sheet = True
        sheet.protection.set_password(self.workbook_password)
        sheet.add_data_validation(ADHERENCE_VALIDATION)

    def write_headers(self):
        style = openpyxl.styles.Style(
            alignment=openpyxl.styles.Alignment(horizontal='left'),
            border=openpyxl.styles.Border(top=THIN_BORDER, bottom=THIN_BORDER),
            font=openpyxl.styles.Font(bold=True),
        )
        for index, header in enumerate(self.headers):
            cell = self.sheet.cell(row=1, column=index + 1)
            cell.value = header
            cell.style = style
            column_letter = openpyxl.cell.get_column_letter(index + 1)
            self.sheet.column_dimensions[column_letter].width = 18
    
    @property
    def filename_base(self):
        return date.format('YYYY-MM-DD', formatter='alternative')
    
    def save(self, filename=None):
        if filename is None:
            filename = '{}.xlsx'.format(self.filename_base)
        self.workbook.save(filename)
        for adherence in tqdm.tqdm(self.patients_adherence):
            directory = os.path.join(
                self.thermogram_directory,
                adherence.patient.id,
                self.date.format('YYYY-MM-DD', formatter='alternative'),
            )
            self.ensure_directory_exists(directory)
            for scan in adherence.actuated_scans:
                filename = os.path.join(directory, '{}.png'.format(scan.id))
                RawThermogramWriter(scan.raw_measurement_thermograms).save(filename)

In [10]:
START_DATE = pendulum.now(tz='America/New_York') # pendulum.create(2017, 12, 13, tz='America/New_York')#
days = 1
patients = ApiPatients.from_api().filter_by_affiliations('VA Tucson', 'VA Phoenix', 'VA Grand Junction', 'VA Long Beach', 'VA Salt Lake City')
date = START_DATE
writer = WRITER = PatientAdherenceWriter(patients, date, 'HealthyF33t!')

In [11]:
#START_DATE = #pendulum.create(2017, 12, 11, tz='America/New_York')#
#date = START_DATE
#days = 1

In [12]:
while True:
    try:
        date = START_DATE.subtract(days=days).replace(hour=0, minute=0, second=0, microsecond=0)
        filename = '{}.xlsx'.format(date.format('YYYY-MM-DD', formatter='alternative'))
        if os.path.isfile(filename):
            raise ValueError('Date already exists!')
        print(filename)
        writer.date = date.replace(hour=0, minute=0, second=0, microsecond=0)
        writer.cache_patients_adherence(progress=True)
        writer.populate_workbook()
        writer.save(filename)
        days += 1
        #break    
    except ValueError:
        break

  0%|          | 0/549 [00:00<?, ?it/s]

2018-05-27.xlsx


100%|██████████| 549/549 [00:00<00:00, 42004.25it/s]


2018-05-26.xlsx


100%|██████████| 549/549 [00:00<00:00, 23364.82it/s]


...

100%|██████████| 549/549 [00:00<00:00, 18923.85it/s]


2018-05-22.xlsx


100%|██████████| 549/549 [00:00<00:00, 32734.93it/s]


2018-05-21.xlsx


100%|██████████| 549/549 [00:19<00:00, 28.46it/s]


# Verification

In [ ]:
class SpreadsheetPatientAdherence:
    
    def __init__(
            self, date, patient_name, patient_affiliation, patient_id, adherence_status,
            thermograms_directory, footness, serial_number, days_since_last_diagnostic,
            review_required, *args, **kwargs
        ):
        self.date = date.date()
        self.patient_name = patient_name
        self.patient_affiliation = patient_affiliation
        self.patient_id = patient_id
        self.adherence_status = adherence_status
        self.footness = footness
        self.serial_number = serial_number
    
    def __repr__(self):
        return '<Patient {} adherence status on {}: {}>'.format(self.date, self.patient_name, self.adherence_status)


class SpreadsheetAdherence:
    
    def __init__(self, filename):
        self.workbook = openpyxl.load_workbook(filename=filename)
        self.sheet = self.workbook['Adherence']
        
    def __iter__(self):
        for row in self.sheet.rows[1:]:
            yield SpreadsheetPatientAdherence(*[column.value for column in row])

In [ ]:
workbook = openpyxl.load_workbook(filename='Patient Monitoring Log.xlsx')
sheet = workbook['Feb, 2017']

In [ ]:
ACHILLES_ADHERENCE_BY_PATIENT = collections.defaultdict(dict)
MANUAL_ADHERENCE_BY_PATIENT = collections.defaultdict(dict)
VERIFICATION_PERIOD = pendulum.period(pendulum.create(2017, 2, 14), pendulum.create(2017, 2, 26))

In [ ]:
for date in VERIFICATION_PERIOD.range('days'):
    spreadsheet_adherence = SpreadsheetAdherence(
        '{}.xlsx'.format(date.format('YYYY-MM-DD', formatter='alternative'))
    )
    for patient_adherence in spreadsheet_adherence:
        if patient_adherence.serial_number is None:
            continue
        patient_name = patient_adherence.patient_name
        ACHILLES_ADHERENCE_BY_PATIENT[patient_name][date.date()] = patient_adherence.adherence_status

In [ ]:
PATIENT_COLUMN_START = openpyxl.utils.column_index_from_string('H')
PATIENT_COLUMN_STOP = openpyxl.utils.column_index_from_string('MF')
ADHERENCE_MAPPING = {
    None: False,
    'NONE': False,
    'OK': True,
    'PRE-PR': True,
    'PROTOCOL': True,
}


def process_name(name):
    if 'Tuscon' in name:
        name = name.replace('Tuscon', 'Tucson')
    if name == 'S-XXX':
        name = 'S-XXXX'
    if name == 'David Faulker':
        name = 'David Faulkner'
    if name == 'A-9871':
        name = 'A-9817'
    if name == 'Fredrick   Mclouth':
        name = 'Fredrick Mclouth'
    if name == 'G1598':
        name = 'G-1598'
    if name == 'Joyce \nStrickland':
        name = 'Joyce Strickland'
    if name == 'K-1326':
        name = 'K-1362'
    if name == 'RusselL\nMurray':
        name = 'Russell Murray'
    if name == 'Sherman Zachary':
        name = 'Shermon Zachary'
    if name == 'Sherron \nWestor':
        name = 'Sherron Wester'
    if name == 'Y-6229':
        name = 'Y-6299'
    return name


for row_index in range(16, 29):
    date = sheet.cell(row=row_index, column=1).value
    for column_index in range(PATIENT_COLUMN_START, PATIENT_COLUMN_STOP + 1):
        patient_name = process_name(sheet.cell(row=2, column=column_index).value)
        adherence_status = ADHERENCE_MAPPING[sheet.cell(row=row_index, column=column_index).value]
        MANUAL_ADHERENCE_BY_PATIENT[patient_name][date.date()] = adherence_status

In [ ]:
MANUAL_ADHERENCE_BY_PATIENT['S-5618'] == ACHILLES_ADHERENCE_BY_PATIENT['S-5618']

In [ ]:
set(MANUAL_ADHERENCE_BY_PATIENT).difference(ACHILLES_ADHERENCE_BY_PATIENT)

In [ ]:
# these should have no scans
patients_unique_to_achilles = set(ACHILLES_ADHERENCE_BY_PATIENT).difference(MANUAL_ADHERENCE_BY_PATIENT)
# patients shared
patients_shared = set(ACHILLES_ADHERENCE_BY_PATIENT).intersection(MANUAL_ADHERENCE_BY_PATIENT)

In [ ]:
TP = 0
P = 0
FP = 0
N = 0
for patient in patients_shared:
    manual_adherence = MANUAL_ADHERENCE_BY_PATIENT[patient]
    achilles_adherence = ACHILLES_ADHERENCE_BY_PATIENT[patient]
    for date in VERIFICATION_PERIOD.range('days'):
        if date.date() not in achilles_adherence:
            continue
        if manual_adherence[date.date()]:
            P += 1
        else:
            N += 1
        if achilles_adherence[date.date()] and manual_adherence[date.date()]:
            TP += 1
        if achilles_adherence[date.date()] and not manual_adherence[date.date()]:
            FP += 1
        """
        if manual_adherence[date.date()] != achilles_adherence[date.date()]:
            print(patient, date.date(), manual_adherence[date.date()], achilles_adherence[date.date()])
            cntr += 1
        """

In [ ]:
TP/P, FP/N

In [ ]:
len(MANUAL_ADHERENCE_BY_PATIENT)*VERIFICATION_PERIOD.days

In [ ]:
VERIFICATION_PERIOD.days